In [1]:
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader

# 설정
pdf_path = "course/rawdata/국어 지식 기반 생성(RAG) 참조 문서.pdf"
output_dir = Path("rag_knowledge_base/chunks")
output_dir.mkdir(parents=True, exist_ok=True)

UNNEEDED_FIELDS = {"category", "sub_category", "title", "source", "error_types", "linked_rules"}

# PDF 파싱
reader = PdfReader(pdf_path)
full_text = "\n".join([page.extract_text() for page in reader.pages])
chunks = re.split(r'(?=<[^<>]+>)', full_text)

category_index = {}
category_chunks = {}

for raw_chunk in chunks:
    raw_chunk = raw_chunk.strip()
    if not raw_chunk.startswith("<"):
        continue

    match = re.match(r"<([^<>]+)>", raw_chunk)
    if not match:
        continue

    title = match.group(1).strip()
    body = raw_chunk[match.end():].strip()

    chunk_title = title.split("-")[-1].strip() if "-" in title else title
    overview = body.split("\n")[0].strip() if "\n" in body else body.strip()

    category = title.split("-")[0].strip() if "-" in title else "기타"
    prefix = category.replace(" ", "_").lower()
    category_index.setdefault(category, 0)
    category_index[category] += 1
    idx = category_index[category]
    chunk_id = f"{prefix}_{idx:02d}"

    # 최종 저장할 구조
    entry = {
        "id": chunk_id,
        "rule": overview,
        "conditions": [],
        "examples": {
            "correct": [],
            "incorrect": []
        },
        "keywords": [],
        "content": body
    }

    category_chunks.setdefault(category, []).append(entry)

# 저장
print("📚 추출된 규범 카테고리:")
for category, data in category_chunks.items():
    print(f"- {category} ({len(data)} 청크)")
    
    filename = category.replace(" ", "_").replace("/", "_") + ".json"
    with open(output_dir / filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ 최종 청크 파일 저장 완료")

📚 추출된 규범 카테고리:
- 띄어쓰기 (10 청크)
- 문장 부호 (17 청크)
- 외래어  표기법 (23 청크)
- 한글 맞춤법 , 표준어  규정 (65 청크)
✅ 최종 청크 파일 저장 완료


In [1]:
import json
import os
from openai import OpenAI
from pathlib import Path
from dotenv import load_dotenv
from tqdm import tqdm

# ✅ .env에서 OPENAI_API_KEY 불러오기
load_dotenv()

# ✅ chunks 디렉토리 경로
chunk_dir = Path("rag_knowledge_base/chunks")
json_files = list(chunk_dir.glob("*.json"))

# ✅ 프롬프트 템플릿
PROMPT_TEMPLATE = """
다음은 한국어 어문 규범 설명입니다. 이 규칙에 기반하여 아래 항목들을 작성해주세요.

1. 이 규칙이 적용되는 조건을 bullet 형식으로 정리해주세요 (설명 안에서 유추 가능한 수준으로만).
2. 규범 설명에 포함된 표현이나 예시를 바탕으로, 올바른 예와 잘못된 예를 각각 제시해주세요. 새로운 문장은 만들지 말고, 설명에 포함된 어구만 사용하세요.
3. 규범에서 중심 개념이나 판단 기준이 되는 단어나 표현을 핵심 키워드로 추출해주세요.

다음 JSON 포맷으로 출력해주세요:
{{
  "conditions": [...],
  "examples": {{
    "correct": [...],
    "incorrect": [...]
  }},
  "keywords": [...]
}}

--- 규범 설명 ---
{rule}

{content}
"""


client = OpenAI()

def call_openai(prompt, model="gpt-4", max_tokens=512):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

# ✅ 전체 파일 순회
for json_file in json_files:
    print(f"\n📂 처리 중: {json_file.name}")
    with open(json_file, "r", encoding="utf-8") as f:
        chunks = json.load(f)

    filled_chunks = []
    for chunk in tqdm(chunks, desc=f"{json_file.name} 처리 중"):
        prompt = PROMPT_TEMPLATE.format(rule=chunk.get("rule", ""), content=chunk.get("content", ""))
        
        try:
            result = json.loads(call_openai(prompt))
            chunk["conditions"] = result.get("conditions", [])
            chunk["examples"] = result.get("examples", {"correct": [], "incorrect": []})
            chunk["keywords"] = result.get("keywords", [])
        except Exception as e:
            print(f"⚠️ 오류 발생 - {chunk.get('id', 'unknown')}: {e}")
        
        filled_chunks.append(chunk)

    # ✅ _v2 파일로 저장
    output_file = json_file.with_name(json_file.stem + "_v2.json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(filled_chunks, f, ensure_ascii=False, indent=2)

    print(f"✅ 저장 완료 → {output_file.name}")


📂 처리 중: 띄어쓰기.json


띄어쓰기.json 처리 중: 100%|██████████| 10/10 [01:55<00:00, 11.60s/it]


✅ 저장 완료 → 띄어쓰기_v2.json

📂 처리 중: 문장_부호.json


문장_부호.json 처리 중:  12%|█▏        | 2/17 [01:12<09:27, 37.83s/it]

⚠️ 오류 발생 - 문장_부호_02: Unterminated string starting at: line 43 column 5 (char 773)


문장_부호.json 처리 중:  18%|█▊        | 3/17 [01:37<07:31, 32.24s/it]

⚠️ 오류 발생 - 문장_부호_03: Unterminated string starting at: line 18 column 7 (char 609)


문장_부호.json 처리 중:  18%|█▊        | 3/17 [02:01<09:28, 40.60s/it]


KeyboardInterrupt: 